In [ ]:
from google.colab import drive
import os

drive.mount("/content/gdrive")
path = "/content/gdrive/My Drive/Age_Gender_Classification"
os.chdir(path)
!ls


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
age_acc_RMSprop_e20_1024_512.png     loss.png
Age_Gender_Classification.ipynb      MobileNet_model_RMSprop_e20_1024_512.h5
checkpoints			     temp
count.ipynb			     test
gender_acc_RMSprop_e20_1024_512.png  weight_RMSprop_e20_1024_512.hdf5
haarcascade_frontalface_default.xml


In [ ]:
!git clone https://github.com/minhnhat97kg/uktface.git

Cloning into 'uktface'...
remote: Enumerating objects: 23330, done.
remote: Total 23330 (delta 0), reused 0 (delta 0), pack-reused 23330
Receiving objects: 100% (23330/23330), 110.09 MiB | 11.52 MiB/s, done.
Checking out files: 100% (23710/23710), done.


In [ ]:
!mv uktface/UTKFace ./UTKFace

In [ ]:
!pwd

/content/gdrive/My Drive/Age_Gender_Classification


In [ ]:
import numpy as np
import pandas as pd 
import cv2
from IPython.display import Image
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import glob
import random

In [ ]:
def imshow(img):
    plt.imshow(img, interpolation='bicubic')
    plt.xticks([]), plt.yticks([]) 
    plt.show()

In [ ]:
paths = glob.glob('UTKFace/*.jpg')
onlyfiles = [ i.split("/")[-1] for i in paths]
y = np.array([[[float(i.split('_')[0])],[float(i.split('_')[1])]] for i in onlyfiles])
#y = np.array([[i.split('_')[1] for i in onlyfiles]]).T
print(y.shape)
print(y[22])

(23708, 2, 1)
[[10.]
 [ 0.]]


In [ ]:
male = [0]*120
female = [0]*120

In [ ]:
for x in y:
  if x[1] == 0:
    z = int(x[0])
    female[z] = female[z] + 1
  else:
    male[z] = male[z] + 1


In [79]:
import pandas as pd


In [90]:
df = pd.DataFrame() 
  
# Creating two columns 
df['Male'] = male[0:] 
df['Female'] = female[0:] 
  
# Converting to excel 
df.to_excel('result.xlsx', index = True, index_label='Age') 

In [ ]:
for i in range(len(male)):
  print("Tuoi:", i, " SL:", male[i])

Tuoi: 0  SL: 0
Tuoi: 1  SL: 478
Tuoi: 2  SL: 254
Tuoi: 3  SL: 192
Tuoi: 4  SL: 165
Tuoi: 5  SL: 118
Tuoi: 6  SL: 102
Tuoi: 7  SL: 80
Tuoi: 8  SL: 196
Tuoi: 9  SL: 30
Tuoi: 10  SL: 65
Tuoi: 11  SL: 39
Tuoi: 12  SL: 87
Tuoi: 13  SL: 43
Tuoi: 14  SL: 82
Tuoi: 15  SL: 94
Tuoi: 16  SL: 114
Tuoi: 17  SL: 83
Tuoi: 18  SL: 185
Tuoi: 19  SL: 58
Tuoi: 20  SL: 185
Tuoi: 21  SL: 227
Tuoi: 22  SL: 260
Tuoi: 23  SL: 286
Tuoi: 24  SL: 595
Tuoi: 25  SL: 446
Tuoi: 26  SL: 1242
Tuoi: 27  SL: 316
Tuoi: 28  SL: 440
Tuoi: 29  SL: 285
Tuoi: 30  SL: 377
Tuoi: 31  SL: 180
Tuoi: 32  SL: 334
Tuoi: 33  SL: 81
Tuoi: 34  SL: 156
Tuoi: 35  SL: 343
Tuoi: 36  SL: 195
Tuoi: 37  SL: 111
Tuoi: 38  SL: 163
Tuoi: 39  SL: 120
Tuoi: 40  SL: 180
Tuoi: 41  SL: 45
Tuoi: 42  SL: 94
Tuoi: 43  SL: 55
Tuoi: 44  SL: 30
Tuoi: 45  SL: 107
Tuoi: 46  SL: 54
Tuoi: 47  SL: 44
Tuoi: 48  SL: 48
Tuoi: 49  SL: 69
Tuoi: 50  SL: 98
Tuoi: 51  SL: 41
Tuoi: 52  SL: 63
Tuoi: 53  SL: 97
Tuoi: 54  SL: 84
Tuoi: 55  SL: 57
Tuoi: 56  SL: 99
Tuoi: 57  S

In [ ]:
dataset_folder_name = 'UTKFace'

TRAIN_TEST_SPLIT = 0.7
IM_WIDTH = IM_HEIGHT = 198

dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black', 
        2: 'asian', 
        3: 'indian', 
        4: 'others'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((g, i) for i, g in dataset_dict['race_id'].items())

In [ ]:
import numpy as np 
import pandas as pd
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def parse_dataset(dataset_path, ext='jpg'):
    """
    Used to extract information about our dataset. It does iterate over all images and return a DataFrame with
    the data (age, gender and sex) of all files.
    """
    def parse_info_from_file(path):
        """
        Parse information from a single file
        """
        try:
            filename = os.path.split(path)[1]
            filename = os.path.splitext(filename)[0]
            age, gender, race, _ = filename.split('_')

            return int(age), dataset_dict['gender_id'][int(gender)], dataset_dict['race_id'][int(race)]
        except Exception as ex:
            return None, None, None
        
    files = glob.glob(os.path.join(dataset_path, "*.%s" % ext))
    
    records = []
    for file in files:
        info = parse_info_from_file(file)
        records.append(info)
        
    df = pd.DataFrame(records)
    df['file'] = files
    df.columns = ['age', 'gender', 'race', 'file']
    df = df.dropna()
    
    return df

In [ ]:
df = parse_dataset(dataset_folder_name)
df.head()

,age,gender,race,file
0,100.0,male,white,UTKFace/100_0_0_20170112213500903.jpg.chip.jpg
1,100.0,male,white,UTKFace/100_0_0_20170112215240346.jpg.chip.jpg
2,100.0,female,white,UTKFace/100_1_0_20170110183726390.jpg.chip.jpg
3,100.0,female,white,UTKFace/100_1_0_20170112213001988.jpg.chip.jpg
4,100.0,female,white,UTKFace/100_1_0_20170112213303693.jpg.chip.jpg


In [ ]:
import plotly.graph_objects as go
from IPython.display import display, Image

def plot_distribution(pd_series):
    labels = pd_series.value_counts().index.tolist()
    counts = pd_series.value_counts().values.tolist()
    
    pie_plot = go.Pie(labels=labels, values=counts, hole=.3)
    fig = go.Figure(data=[pie_plot])
    fig.update_layout(title_text='Distribution for %s' % pd_series.name)
        
    img_bytes = fig.to_image(format="png")
    display(Image(img_bytes))

In [ ]:
plot_distribution(df['gender'])

ValueError: ignored